## Setting up OpenAI Model

In [1]:
# Install llama-index libraries
!pip install llama-index
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-openai
!pip install llama-index-embeddings-huggingface

# Install wikipedia python package
!pip install wikipedia
!pip install llama-index-readers-wikipedia

In [2]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "ADD_KEY_HERE"

In [3]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# Vectors will be stored in pinecone.io

from pinecone import Pinecone, ServerlessSpec
import pinecone

api_key= "ADD_KEY_HERE"
pc = Pinecone(api_key=api_key)
pc.create_index("quickstart", dimension=1536, metric="euclidean",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"))

pinecone_index = pc.Index("quickstart")

In [5]:
# define pinecone vector index

from llama_index.core import StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

## Setting up Text to SQL

In [6]:
# sql library
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    select,
    column,
)
# create engine
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [7]:
# Creating aluminum sql table
# Source = https://www.stressebook.com/mmpds-mechanical-properties-table/

table_name = "aluminum_2024_material_allowables"
material_allowable_table = Table(
    table_name,
    metadata_obj,
    Column("material_name", String(16), primary_key=True),
    Column("specification", String(16), primary_key=True),
    Column("form", String(16), primary_key=True),
    Column("basis", String(16), primary_key=True),
    Column("stock_thickness", String(16), primary_key=True),
    Column("Ftu_ksi_L", Integer, primary_key=True),
    Column("Ftu_ksi_LT", Integer, primary_key=True),
    Column("Ftu_ksi_ST", Integer, primary_key=True),
    Column("Fty_ksi_L", Integer, primary_key=True),
    Column("Fty_ksi_LT", Integer, primary_key=True),
    Column("Fty_ksi_ST", Integer, primary_key=True),
    Column("Fcy_ksi_L", Integer, primary_key=True),
    Column("Fcy_ksi_LT", Integer, primary_key=True),
    Column("Fcy_ksi_ST", Integer, primary_key=True),
    Column("Fsu_ksi_L_LT", Integer, primary_key=True),
    Column("Fbru_ksi_L_LT_eD_1_5", Integer, primary_key=True),
    Column("Fbru_ksi_L_LT_eD_2_0", Integer, primary_key=True),
    Column("Fbry_ksi_L_LT_eD_1_5", Integer, primary_key=True),
    Column("Fbry_ksi_L_LT_eD_2_0", Integer, primary_key=True),
    Column("e_percent_S_Basis", Integer, primary_key=True),
    Column("Elastic_Modulus_ksi", String(16), primary_key=True),
)

metadata_obj.create_all(engine)

In [8]:
# Insert rows

from sqlalchemy import insert

rows = [
    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "A",
     "stock_thickness": "0.25 - 0.49",
     "Ftu_ksi_L": 64,
     "Ftu_ksi_LT": 64,
     "Ftu_ksi_ST": 64,
     "Fty_ksi_L": 48,
     "Fty_ksi_LT": 42,
     "Fty_ksi_ST": 48,
     "Fcy_ksi_L": 39,
     "Fcy_ksi_LT": 45,
     "Fcy_ksi_ST": 45,
     "Fsu_ksi_L_LT": 38,
     "Fbru_ksi_L_LT_eD_1_5": 97,
     "Fbru_ksi_L_LT_eD_2_0": 119,
     "Fbry_ksi_L_LT_eD_1_5": 72,
     "Fbry_ksi_L_LT_eD_2_0": 86,
     "e_percent_S_Basis": 12,
     "Elastic_Modulus_ksi": "10.7"},

    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "B",
     "stock_thickness": "0.25 - 0.49",
     "Ftu_ksi_L": 66,
     "Ftu_ksi_LT": 66,
     "Ftu_ksi_ST": 66,
     "Fty_ksi_L": 50,
     "Fty_ksi_LT": 44,
     "Fty_ksi_ST": 50,
     "Fcy_ksi_L": 45,
     "Fcy_ksi_LT": 45,
     "Fcy_ksi_ST": 38,
     "Fsu_ksi_L_LT": 39,
     "Fbru_ksi_L_LT_eD_1_5": 100,
     "Fbru_ksi_L_LT_eD_2_0": 122,
     "Fbry_ksi_L_LT_eD_1_5": 76,
     "Fbry_ksi_L_LT_eD_2_0": 90,
     "e_percent_S_Basis": 12,
     "Elastic_Modulus_ksi": "10.7"},

    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "A",
     "stock_thickness": "0.5 - 0.1",
     "Ftu_ksi_L": 63,
     "Ftu_ksi_LT": 63,
     "Ftu_ksi_ST": 63,
     "Fty_ksi_L": 48,
     "Fty_ksi_LT": 42,
     "Fty_ksi_ST": 48,
     "Fcy_ksi_L": 39,
     "Fcy_ksi_LT": 45,
     "Fcy_ksi_ST": 45,
     "Fsu_ksi_L_LT": 37,
     "Fbru_ksi_L_LT_eD_1_5": 95,
     "Fbru_ksi_L_LT_eD_2_0": 117,
     "Fbry_ksi_L_LT_eD_1_5": 72,
     "Fbry_ksi_L_LT_eD_2_0": 86,
     "e_percent_S_Basis": 8,
     "Elastic_Modulus_ksi": "10.7"},

    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "B",
     "stock_thickness": "0.5 - 0.1",
     "Ftu_ksi_L": 65,
     "Ftu_ksi_LT": 65,
     "Ftu_ksi_ST": 65,
     "Fty_ksi_L": 50,
     "Fty_ksi_LT": 44,
     "Fty_ksi_ST": 50,
     "Fcy_ksi_L": 41,
     "Fcy_ksi_LT": 47,
     "Fcy_ksi_ST": 47,
     "Fsu_ksi_L_LT": 38,
     "Fbru_ksi_L_LT_eD_1_5": 98,
     "Fbru_ksi_L_LT_eD_2_0": 120,
     "Fbry_ksi_L_LT_eD_1_5": 76,
     "Fbry_ksi_L_LT_eD_2_0": 90,
     "e_percent_S_Basis": 8,
     "Elastic_Modulus_ksi": "10.7"},

    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "A",
     "stock_thickness": "0.1 - 1.5",
     "Ftu_ksi_L": 62,
     "Ftu_ksi_LT": 62,
     "Ftu_ksi_ST": 62,
     "Fty_ksi_L": 47,
     "Fty_ksi_LT": 42,
     "Fty_ksi_ST": 47,
     "Fcy_ksi_L": 39,
     "Fcy_ksi_LT": 44,
     "Fcy_ksi_ST": 44,
     "Fsu_ksi_L_LT": 37,
     "Fbru_ksi_L_LT_eD_1_5": 94,
     "Fbru_ksi_L_LT_eD_2_0": 115,
     "Fbry_ksi_L_LT_eD_1_5": 72,
     "Fbry_ksi_L_LT_eD_2_0": 86,
     "e_percent_S_Basis": 7,
     "Elastic_Modulus_ksi": "10.7"},

    {"material_name": "Aluminum Alloy",
     "specification": "AMS 4037 and AMS-QQ-A-250/4",
     "form": "Plate",
     "basis": "B",
     "stock_thickness": "0.1 - 1.5",
     "Ftu_ksi_L": 64,
     "Ftu_ksi_LT": 64,
     "Ftu_ksi_ST": 64,
     "Fty_ksi_L": 50,
     "Fty_ksi_LT": 44,
     "Fty_ksi_ST": 50,
     "Fcy_ksi_L": 40,
     "Fcy_ksi_LT": 46,
     "Fcy_ksi_ST": 46,
     "Fsu_ksi_L_LT": 38,
     "Fbru_ksi_L_LT_eD_1_5": 97,
     "Fbru_ksi_L_LT_eD_2_0": 119,
     "Fbry_ksi_L_LT_eD_1_5": 76,
     "Fbry_ksi_L_LT_eD_2_0": 90,
     "e_percent_S_Basis": 7,
     "Elastic_Modulus_ksi": "10.7"}
]


for row in rows:
    stmt = insert(material_allowable_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [9]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM aluminum_2024_material_allowables")
    print(cursor.fetchall())

[('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'A', '0.25 - 0.49', 64, 64, 64, 48, 42, 48, 39, 45, 45, 38, 97, 119, 72, 86, 12, '10.7'), ('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'B', '0.25 - 0.49', 66, 66, 66, 50, 44, 50, 45, 45, 38, 39, 100, 122, 76, 90, 12, '10.7'), ('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'A', '0.5 - 0.1', 63, 63, 63, 48, 42, 48, 39, 45, 45, 37, 95, 117, 72, 86, 8, '10.7'), ('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'B', '0.5 - 0.1', 65, 65, 65, 50, 44, 50, 41, 47, 47, 38, 98, 120, 76, 90, 8, '10.7'), ('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'A', '0.1 - 1.5', 62, 62, 62, 47, 42, 47, 39, 44, 44, 37, 94, 115, 72, 86, 7, '10.7'), ('Aluminum Alloy', 'AMS 4037 and AMS-QQ-A-250/4', 'Plate', 'B', '0.1 - 1.5', 64, 64, 64, 50, 44, 50, 40, 46, 46, 38, 97, 119, 76, 90, 7, '10.7')]


## Set up Semantic Data

In [10]:
from llama_index.readers.wikipedia import WikipediaReader

materials = ["2024 Aluminum Alloy Plate"]
wiki_docs = WikipediaReader().load_data(pages=materials)

## SQL Index

In [11]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["aluminum_2024_material_allowables"])

In [12]:
# Table Query Engine

from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["aluminum_2024_material_allowables"],
)

## Semantic Data Index

In [13]:
# Use HuggingFace Embedding Model

from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# Insert documents into vector index
for mat, w_doc in zip(materials, wiki_docs):
    nodes = Settings.node_parser.get_nodes_from_documents([w_doc])
    # Add metadata to each node
    for node in nodes:
        node.metadata = {"title": mat}
    vector_index.insert_nodes(nodes)

Upserted vectors:   0%|          | 0/7 [00:00<?, ?it/s]

## Combine SQL and Semantic text Query Engines

In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core.query_engine import RetrieverQueryEngine


vector_store_info = VectorStoreInfo(
    content_info="articles about aluminum material",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="The name of the material"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=OpenAI(model="gpt-4o-mini")
)

In [16]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: material_name, specification, form, basis, stock_thickness, Ftu_ksi_L, Ftu_ksi_LT, Ftu_ksi_ST,"
        " Fty_ksi_L, Fty_ksi_LT, Fty_ksi_ST, Fcy_ksi_L, Fcy_ksi_LT, Fcy_ksi_ST,"
        " Fsu_ksi_L_LT, Fbru_ksi_L_LT_eD_1_5, Fbru_ksi_L_LT_eD_2_0,"
        " Fbry_ksi_L_LT_eD_1_5, Fbry_ksi_L_LT_eD_2_0, e_percent_S_Basis, Elastic_Modulus_ksi of"
        " each material"
    ),
)


vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about 2024 aluminum plate."
    ),
)

In [17]:
from llama_index.core.query_engine import SQLAutoVectorQueryEngine

query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, llm=OpenAI(model="gpt-4o-mini")
)

In [18]:
response = query_engine.query(
    "Tell me about aluminum and its lowest ftu number at stock thickness 0.25 - 0.49"
)

Querying SQL database: The first choice is relevant because it provides a way to translate a natural language query into a SQL query that can retrieve specific data about aluminum, including its properties and the lowest Ftu number at the specified stock thickness.
SQL query: SELECT material_name, Ftu_ksi_L
FROM aluminum_2024_material_allowables
WHERE stock_thickness = '0.25 - 0.49'
ORDER BY Ftu_ksi_L
LIMIT 1;
SQL response: The aluminum alloy with the lowest ultimate tensile strength (UTS) at a stock thickness of 0.25 - 0.49 inches is 64 ksi.
Transformed query given SQL response: What is the specific aluminum alloy that has the lowest ultimate tensile strength (UTS) of 64 ksi at a stock thickness of 0.25 - 0.49 inches?
query engine response: The specific aluminum alloy that has the lowest ultimate tensile strength (UTS) of 64 ksi at a stock thickness of 0.25 - 0.49 inches is 2024-T3.


In [19]:
print(str(response))

The aluminum alloy with the lowest ultimate tensile strength (UTS) at a stock thickness of 0.25 - 0.49 inches is 2024-T3, which has a UTS of 64 ksi.


In [20]:
response2 = query_engine.query(
    "Tell me about aluminum and its lowest ftu b-basis number at stock thickness 0.25 - 0.49"
)

Querying SQL database: The question involves translating a natural language query into a SQL query to retrieve specific data about aluminum, including its lowest ftu b-basis number at a specified stock thickness.
SQL query: SELECT material_name, Ftu_ksi_LT
FROM aluminum_2024_material_allowables
WHERE material_name LIKE '%aluminum%' AND basis = 'B' AND stock_thickness = '0.25 - 0.49'
ORDER BY Ftu_ksi_LT
SQL response: The aluminum alloy with the lowest Ftu B-basis number at a stock thickness of 0.25 - 0.49 is 66 ksi.
Transformed query given SQL response: What is the specific aluminum alloy that has the lowest Ftu B-basis number at a stock thickness of 0.25 - 0.49?
query engine response: The specific aluminum alloy that has the lowest Ftu B-basis number at a stock thickness of 0.25 - 0.49 is 2024 aluminum alloy.


In [22]:
print(str(response2))

The aluminum alloy with the lowest Ftu B-basis number at a stock thickness of 0.25 - 0.49 is the 2024 aluminum alloy, which has a Ftu B-basis number of 66 ksi.


In [23]:
response3 = query_engine.query(
    "Tell me the lowest ftu allowable for aluminum 2024 plate at stock thickness equal to 0.5 - 1.0"
)

Querying SQL database: The first choice is relevant because it provides a way to translate a natural language query into a SQL query that can retrieve specific data about material properties, including the Ftu allowable for aluminum 2024 plate.
SQL query: SELECT material_name, stock_thickness, Ftu_ksi_L
FROM aluminum_2024_material_allowables
WHERE material_name = 'aluminum 2024' AND form = 'plate' AND stock_thickness = '0.5 - 1.0'
ORDER BY Ftu_ksi_L
LIMIT 1;
SQL response: I'm sorry, but there are no specific results available for the lowest allowable Ftu for aluminum 2024 plate at a stock thickness of 0.5 - 1.0. It seems that the data for this specific combination is not available in the database.
Transformed query given SQL response: What are the allowable Ftu values for aluminum 2024 plate at different stock thicknesses?
query engine response: Empty Response


In [24]:
print(str(response3))

It appears that there is no available data for the lowest allowable Ftu (tensile strength) for aluminum 2024 plate at a stock thickness of 0.5 - 1.0. Both the SQL query and the vector store query returned no results, indicating that this specific information may not be present in the databases consulted. If you have any other questions or need information on different materials or thicknesses, feel free to ask!
